In [6]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

# Exercice N1

In [21]:
#Loading the file
loadingrequest = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(loadingrequest.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]


In [22]:
#The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [23]:
#initial shape
df.shape

(180, 3)

In [24]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
index = df[ df['Borough'] == "Not assigned" ].index
df.drop(index , inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [25]:
#After droping shape
df.shape

(103, 3)

In [26]:
#More than one neighborhood can exist in one postal code area. For example, 
#in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: 
#Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma
#as shown in row 11 in the above table.

df_duplicate = df[df.duplicated()]
print(df_duplicate)
#There is no duplicate in the whole dataframe => Nothing to be processed



Empty DataFrame
Columns: [Postal Code, Borough, Neighborhood]
Index: []


In [27]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
pd.isnull(df['Neighborhood']).value_counts()
#There no Not assigned neighborhood => nothing to be processed

False    103
Name: Neighborhood, dtype: int64

In [28]:
#In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.shape

(103, 3)

# Exercice N2

In [29]:
latlon=pd.read_csv("http://cocl.us/Geospatial_data")
latlon.shape
latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
#Merging dataframe based on Postal Code
df=pd.merge(df, latlon, on=['Postal Code'])
df.head()



,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [31]:
#After merging shape
df.shape

(103, 5)

# Exercice N3

In [58]:
#Focussing on Toronto neighborhood
toronto_data = df[df.Borough.str.contains('Toronto')].reset_index(drop=True)
toronto_data.drop(['Postal Code'], axis=1,inplace=True)

#### Set up the connexion

In [59]:
CLIENT_ID = 'N5SWU5EU5F0X0GM2D5KNOG52NTQC4IVKO2GM10RJQA2BMU3H' # your Foursquare ID
CLIENT_SECRET = 'CVSIFR0EDKGMQCQJYI02BY05SY220FWCO41IIOH3UGGWNDRD' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N5SWU5EU5F0X0GM2D5KNOG52NTQC4IVKO2GM10RJQA2BMU3H
CLIENT_SECRET:CVSIFR0EDKGMQCQJYI02BY05SY220FWCO41IIOH3UGGWNDRD


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [62]:
print(toronto_venues.groupby('Neighborhood').count())
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

                                                    Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            30   
Brockton, Parkdale Village, Exhibition Place                           23   
Business reply mail Processing Centre                                  16   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     30   
Christie                                                               17   
Church and Wellesley                                                   30   
Commerce Court, Victoria Hotel                                         30   
Davisville                                                             30   
Davisville North                                                        8   
Dufferin, Dovercourt Village                                           15   

In [63]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,Art Gallery,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(39, 190)

#### Let's print each neighborhood along with the top 5 most common venues

In [65]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                     venue  freq
0       Seafood Restaurant  0.07
1             Cocktail Bar  0.07
2                 Beer Bar  0.07
3              Coffee Shop  0.07
4  Comfort Food Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                  venue  freq
0                  Café  0.13
1           Coffee Shop  0.09
2        Breakfast Spot  0.09
3  Gym / Fitness Center  0.04
4    Italian Restaurant  0.04


----Business reply mail Processing Centre----
              venue  freq
0       Yoga Studio  0.06
1     Auto Workshop  0.06
2        Comic Shop  0.06
3       Pizza Place  0.06
4  Recording Studio  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.19
1       Airport Lounge  0.12
2              Airport  0.06
3             Boutique  0.06
4  Rental Car Location  0.06


----Central Bay Street----
              venue  fr

In [66]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [67]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Coffee Shop,Cocktail Bar,Beer Bar,Comfort Food Restaurant,Park,Restaurant,Bistro,Café,Basketball Stadium
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Performing Arts Venue,Restaurant,Burrito Place,Intersection,Bar,Bakery,Stadium
2,Business reply mail Processing Centre,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Burrito Place,Light Rail Station,Recording Studio,Restaurant,Brewery
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Boat or Ferry,Boutique,Harbor / Marina,Rental Car Location,Coffee Shop,Sculpture Garden,Bar,Airport Terminal
4,Central Bay Street,Coffee Shop,Café,Yoga Studio,Japanese Restaurant,Hotel,Modern European Restaurant,Ice Cream Shop,Sandwich Place,Italian Restaurant,Bubble Tea Shop


Run *k*-means to cluster the neighborhood into 5 clusters.

In [68]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering.head()
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [69]:
toronto_grouped_clustering.shape

(39, 189)

In [70]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Breakfast Spot,Theater,Bakery,Yoga Studio,Restaurant,Pub,Café,Performing Arts Venue
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Italian Restaurant,Sandwich Place,Juice Bar,Distribution Center,Burrito Place,Diner,Beer Bar
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Café,Coffee Shop,Theater,Sandwich Place,Ramen Restaurant,Electronics Store,Shopping Mall,Hotel,Mexican Restaurant,Diner
3,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Gastropub,Café,Coffee Shop,Optical Shop,Diner,Hotel,Italian Restaurant,Japanese Restaurant,Restaurant,Jazz Club
4,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Trail,Health Food Store,Creperie,Donut Shop,Dog Run,Distribution Center,Diner,Dessert Shop,Department Store


In [71]:
address = 'Toronto'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [72]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Park,Breakfast Spot,Theater,Bakery,Yoga Studio,Restaurant,Pub,Café,Performing Arts Venue
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Yoga Studio,Italian Restaurant,Sandwich Place,Juice Bar,Distribution Center,Burrito Place,Diner,Beer Bar
2,"Garden District, Ryerson",Café,Coffee Shop,Theater,Sandwich Place,Ramen Restaurant,Electronics Store,Shopping Mall,Hotel,Mexican Restaurant,Diner
3,St. James Town,Gastropub,Café,Coffee Shop,Optical Shop,Diner,Hotel,Italian Restaurant,Japanese Restaurant,Restaurant,Jazz Club
4,The Beaches,Pub,Trail,Health Food Store,Creperie,Donut Shop,Dog Run,Distribution Center,Diner,Dessert Shop,Department Store
5,Berczy Park,Seafood Restaurant,Coffee Shop,Cocktail Bar,Beer Bar,Comfort Food Restaurant,Park,Restaurant,Bistro,Café,Basketball Stadium
6,Central Bay Street,Coffee Shop,Café,Yoga Studio,Japanese Restaurant,Hotel,Modern European Restaurant,Ice Cream Shop,Sandwich Place,Italian Restaurant,Bubble Tea Shop
7,Christie,Grocery Store,Café,Park,Candy Store,Nightclub,Italian Restaurant,Diner,Restaurant,Baby Store,Athletics & Sports
8,"Richmond, Adelaide, King",Café,Coffee Shop,Opera House,Japanese Restaurant,Speakeasy,Smoke Shop,Seafood Restaurant,Hotel,Restaurant,Deli / Bodega
9,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Pool,Café,Supermarket,Bank,Bar,Gym / Fitness Center,Grocery Store,Middle Eastern Restaurant


In [73]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",Playground,Park,Wine Bar,Creperie,Donut Shop,Dog Run,Distribution Center,Diner,Dessert Shop,Department Store
33,Rosedale,Park,Trail,Playground,Creperie,Donut Shop,Dog Run,Distribution Center,Diner,Dessert Shop,Department Store


In [76]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,Park,Trail,Sushi Restaurant,Jewelry Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Diner,Dessert Shop


In [77]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Park,Bus Line,Swim School,Donut Shop,Dog Run,Distribution Center,Diner,Dessert Shop,Department Store,Deli / Bodega


In [78]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Home Service,Garden,Wine Bar,Eastern European Restaurant,Donut Shop,Dog Run,Distribution Center,Diner,Dessert Shop,Department Store


### Conclusion

In [57]:
#Out of 4 clusters, there is essentially one big cluster representing café, pub, restaurant area as per cluster 0 
